In [16]:
OPENAI_API_KEY = 'api key' # os.getenv('OPENAI_API_KEY')
NVIDIA_API_KEY = 'api key'
nakd_url = 'https://www.na-kd.com'

# Scraping

In [2]:
import requests
import json
import csv

In [54]:
clothes = []
response = requests.get('https://www.na-kd.com/en/category/summer-clothing?sortBy=popularity&count=500&skip=1&jserror=2')
clothes += response.json()
response = requests.get('https://www.na-kd.com/en/category/summer-clothing?sortBy=popularity&count=1000&skip=500&jserror=2')
clothes += response.json()
response = requests.get('https://www.na-kd.com/en/category/summer-clothing?sortBy=popularity&count=1500&skip=1000&jserror=2')
clothes += response.json()
response = requests.get('https://www.na-kd.com/en/category/summer-clothing?sortBy=popularity&count=1680&skip=1500&jserror=2')
clothes += response.json()

KeyboardInterrupt: 

In [26]:
csv_file = 'clothes_data.csv'
with open(csv_file, mode='w', newline='') as file:
	writer = csv.writer(file)
	writer.writerow(['name', 'url', 'price', 'imgs'])
	for item in clothes:
		name = item['variation']['displayName']['$c']
		url = item['url']
		price = item['sizes'][0]['price']['current']
		imgs = ', '.join(item['imageUrls'])
		writer.writerow([name, url, price, imgs])


# Labeling

In [1]:
import ast
import os
import numpy as np
import pandas as pd
from openai import OpenAI


In [4]:
labels = {
	'Style': [
		'Casual',
		'Formal',
		'Bohemian',
		'Vintage',
		'Sporty',
		'Elegant',
		'Streetwear',
		'Minimalist',
		'Preppy',
		'Chic',
		'Punk',
		'Gothic'
	],
	'Occasion': [
		'Beach',
		'Party',
		'Office',
		'Wedding',
		'Date Night',
		'Casual Outing',
		'Vacation',
		'Festival',
		'Brunch',
		'Evening Gala',
		'Concert',
		'Business Meeting'
	],
	'Season': [
		'Summer',
		'Spring',
		'Autumn',
		'Winter',
		'Transitional',
		'All Seasons',
		'Monsoon',
		'Holiday Season',
		'Festival Season',
		'Rainy Season',
		'Cruise',
		'Desert'
	],
	'Color': [
		'White',
		'Black',
		'Blue',
		'Red',
		'Green',
		'Yellow',
		'Pink',
		'Beige',
		'Grey',
		'Purple',
		'Orange',
		'Brown',
		'Teal',
		'Coral',
		'Navy',
		'Olive',
		'Maroon',
		'Turquoise',
		'Mint',
		'Lavender'
		],
	'Pattern': [
		'Striped',
		'Floral',
		'Solid',
		'Polka Dot',
		'Plaid',
		'Animal Print',
		'Geometric',
		'Abstract',
		'Paisley',
		'Chevron',
		'Houndstooth',
		'Tie-Dye'
	],
	'Material': [
		'Linen',
		'Cotton',
		'Silk',
		'Denim',
		'Knit',
		'Chiffon',
		'Satin',
		'Lace',
		'Wool',
		'Polyester',
		'Velvet',
		'Leather'
	],
	'Fit': [
		'Regular Fit',
		'Slim Fit',
		'Loose Fit',
		'Tailored Fit',
		'Relaxed Fit',
		'Oversized Fit',
		'Skinny Fit',
		'Flowy Fit',
		'Boxy Fit',
		'Bodycon Fit',
		'Straight Fit',
		'Tapered Fit'
	],
	'Length': [
		'Mini',
		'Midi',
		'Maxi',
		'Knee-Length',
		'Ankle-Length',
		'Cropped',
		'Full-Length',
		'Tunic-Length',
		'Thigh-Length',
		'Tea-Length',
		'Calf-Length',
		'Floor-Length'
	],
	'Neckline': [
		'V-Neck',
		'Boat Neck',
		'Round Neck',
		'Square Neck',
		'Scoop Neck',
		'Halter Neck',
		'Off Shoulder',
		'High Neck',
		'Sweetheart Neck',
		'Keyhole Neck',
		'Collared Neck',
		'Cowl Neck'
	],
	'Sleeve Type': [
		'Long Sleeve',
		'Short Sleeve',
		'Sleeveless',
		'Cap Sleeve',
		'Balloon Sleeve',
		'Puff Sleeve',
		'Bell Sleeve',
		'3/4 Sleeve',
		'Flutter Sleeve',
		'Kimono Sleeve',
		'Raglan Sleeve',
		'Dolman Sleeve'
	],
	'Silhouette': [
		'A-Line',
		'Sheath',
		'Fit and Flare',
		'Straight',
		'Mermaid',
		'Empire Waist',
		'Peplum',
		'Wrap',
		'Ball Gown',
		'Shift',
		'Trapeze',
		'Bodycon'
	],
	'Texture': [
		'Smooth',
		'Ribbed',
		'Crinkled',
		'Textured',
		'Matte',
		'Shiny',
		'Quilted',
		'Embroidered',
		'Pleated',
		'Fuzzy',
		'Suede',
		'Sheer'
	],
	'Functionality': [
		'Pockets',
		'Adjustable Straps',
		'Drawstring Waist',
		'Elastic Waistband',
		'Built-in Bra',
		'Convertible',
		'Wrinkle-Resistant',
		'Quick-Dry',
		'Breathable',
		'Water-Resistant',
		'Reversible',
		'UV Protection'
	],
	'Closure Type': [
		'Buttoned',
		'Zippered',
		'Hook and Eye',
		'Tied',
		'Pullover',
		'Snap',
		'Velcro',
		'Lace-Up',
		'Buckle',
		'Magnetic',
		'Toggle',
		'Hidden Zip'
	],
	'Embellishments': [
		'Ruffles',
		'Sequins',
		'Beading',
		'Embroidery',
		'Fringe',
		'Lace',
		'Appliques',
		'Studs',
		'Rhinestones',
		'Tassels',
		'Bows',
		'Pearls'
	],
	'Hemline': [
		'Straight Hem',
		'Curved Hem',
		'Asymmetrical Hem',
		'High-Low Hem',
		'Scalloped Hem',
		'Fringed Hem',
		'Raw Hem',
		'Ruffled Hem',
		'Pleated Hem',
		'Layered Hem',
		'Tapered Hem',
		'Handkerchief Hem'
	], 
	'Clothing Category': [
		'Dress',
		'Skirt',
		'Pants',
		'Shorts',
		'Top',
		'Blouse',
		'Shirt',
		'Jacket',
		'Blazer',
		'Vest',
		'Sweater',
		'Cardigan',
		'Coat',
		'Hoodie',
		'Sweatshirt',
		'Tank Top',
		'Singlet',
		'Camisole',
		'Bodysuit',
		'Jumpsuit',
		'Romper',
		'Leggings',
		'Jeans',
		'Tunic',
		'Cape',
		'Poncho',
		'Scarf',
		'Kimono',
		'Wrap',
		'Polo Shirt',
		'Turtleneck',
		'Henley Shirt'
	]
}
categories = list(labels.keys()) + ['Description']

In [7]:
# load secret key
client = OpenAI(api_key=OPENAI_API_KEY)
nvidia_client = OpenAI(
	api_key=NVIDIA_API_KEY,
	base_url='https://integrate.api.nvidia.com/v1'
)

In [5]:
def list_to_string(l):
	return str(l)


def string_to_list(s):
	return ast.literal_eval(s)


def df_to_dict(row):
	row_dict = {}
	for col in row.index:
		if col in ['url', 'price', 'imgs']:
			continue
		elif col in ['name', 'description', 'Description']:
			row_dict[col] = row[col]
		elif col in ['embedding', 'Embedding']:
			row_dict[col] = string_to_list(row[col]) if isinstance(row[col], str) else []
		else:
			try:
				row_dict[col] = [item.strip() for item in row[col].split(',')] if pd.notna(row[col]) else []
			except Exception:
				print(col)
	return row_dict


def df_to_dict_insert(row):
	row_dict = {}
	for col in row.index:
		if col in 'price':
			row_dict[col] = float(row[col])
		elif col in ['name', 'url', 'description', 'Description', 'description_embedding']:
			row_dict[col] = row[col]
		elif col in ['embedding', 'Embedding']:
			row_dict[col] = string_to_list(row[col]) if isinstance(row[col], str) else []
		else:
			try:
				row_dict[col] = [item.strip() for item in row[col].split(',')] if pd.notna(row[col]) else []
			except Exception:
				print(col)
	return row_dict


def get_description(image_url):
	response = client.chat.completions.create(
		model='gpt-4o-mini',
		response_format={'type': 'json_object'},
		messages=[
			{
				'role': 'system',
				'content': f'''
					Here are categories and options for the categories. You need to return a json object which returns a dictionary for the categories and options which best match the clothing. If one category is not applicaple return the empty list for that category. Feel free to use options which are not given but dont add more categories.
					Additionally give a Description for that clothing. 
					Here is an example: {{'Description': 'Beautiful dress with vivid colors perfect for a summer bruch', 'Style': ['Casual'], ...}}
					categories: {json.dumps(labels)}
					'''
			},
			{
				'role': 'user',
				'content': [
					{
						'type': 'image_url',
						'image_url': {
							'url': image_url,
						},
					},
				],
			}
		],
		max_tokens=500,
	)
	return json.loads(response.choices[0].message.content)


def get_description_for_embedding(item):
	response = client.chat.completions.create(
		model='gpt-4o-mini',
		response_format={'type': 'json_object'},
		messages=[
			{
				'role': 'system',
				'content': '''
				Given a json of a clothing item, return a good description which can be turned into a embedding for similarty search based on user preference.
				json: {
					'Description Embedding': 'The 'Fold Up Denim Shorts' are a perfect blend of casual and chic style ...'
				}
				'''
			},
			{
				'role': 'user',
				'content': json.dumps(item)
			}
		],
		max_tokens=800,
	)
	return json.loads(response.choices[0].message.content)


def embed_item(text, query=False):
	response = nvidia_client.embeddings.create(
		input=[text],
		model='nvidia/nv-embed-v1',
		encoding_format='float',
		extra_body={'input_type': 'query' if query else 'passage', 'truncate': 'NONE'}
	)
	return response.data[0].embedding


In [54]:
csv_file = 'clothes_data_with_descriptions.csv'
df = pd.read_csv(csv_file)

# Get the description from the image
failed = []
for i in range(623, min(624, len(df))):
	image_url = df.at[i, 'imgs'].split(',')[0].strip()  # Use the first image URL
	temp = df.at[i, 'Description']
	if (isinstance(temp, str) and temp): continue
	if np.isnan(temp):
		try:
			description_data = get_description(nakd_url + image_url)
			df.at[i, 'Description'] = description_data.get('Description', '')
		except Exception as e:
			failed.append(i)
			print(i)
			print(e)
		# Add the other category values
		for category in categories: # Exclude 'Description'
			if category != 'Description':
				values = description_data.get(category, [])
				df.at[i, category] = ', '.join(values)

# Delete row 396
# df = df.drop(623).reset_index(drop=True)


updated_csv_file = 'clothes_data_with_descriptions.csv'
df.to_csv(updated_csv_file, index=False)

In [55]:
print(failed)

[]


In [ ]:
# Query for the description embedding given all the data of the row
if 'Description Embedding' not in df.columns:
	df['Description Embedding'] = None
if 'Embedding' not in df.columns:
	df['Embedding'] = None
# Get the embedding for the 'Description Embedding'
for i in range(400, min(799, len(df))):
	description_data = get_description_for_embedding(df_to_dict(df.iloc[i]))
	df.at[i, 'Description Embedding'] = description_data.get('Description Embedding', '')
	df.at[i, 'Embedding'] = list_to_string(embed_item(df.at[i, 'Description Embedding']))
updated_csv_file = 'clothes_data_with_descriptions.csv'
df.to_csv(updated_csv_file, index=False)

# Storing in Milvus


In [3]:
csv_file = 'clothes_data_with_descriptions.csv'
df = pd.read_csv(csv_file)

def rename_columns(df):
	column_mapping = {
		'Description': 'description',
		'Style': 'style',
		'Occasion': 'occasion',
		'Season': 'season',
		'Color': 'color',
		'Pattern': 'pattern',
		'Material': 'material',
		'Fit': 'fit',
		'Length': 'length',
		'Neckline': 'neckline',
		'Sleeve Type': 'sleeve_types',
		'Silhouette': 'silhouette',
		'Texture': 'texture',
		'Functionality': 'functionality',
		'Closure Type': 'closure_type',
		'Embellishments': 'embellishments',
		'Hemline': 'hemline',
		'Clothing Category': 'clothing_category',
		'Description Embedding': 'description_embedding',
		'Embedding': 'embedding'
	}
	df.rename(columns=column_mapping, inplace=True)
	return df

df = rename_columns(df)

clothes200 = []
for i in range(200, min(799, len(df))):
	temp = df_to_dict_insert(df.iloc[i])
	temp['id'] = i
	clothes200.append(temp)

NameError: name 'df_to_dict_insert' is not defined

In [5]:
clothes200[399]

NameError: name 'clothes200' is not defined

In [7]:
from pymilvus import (
	connections,
	utility,
	FieldSchema,
	CollectionSchema,
	DataType,
	Collection,
)


In [8]:
connections.connect('default', host='localhost', port='19530')

In [9]:
fields = [ 
  FieldSchema(name='id', dtype=DataType.INT64, is_primary=True, description='primary id'),
	FieldSchema('name', dtype=DataType.VARCHAR, max_length=1024),
	FieldSchema('url', dtype=DataType.VARCHAR, max_length=1024),
	FieldSchema('price', dtype=DataType.FLOAT),
	FieldSchema('imgs', dtype=DataType.ARRAY, element_type=DataType.VARCHAR, max_capacity=32, max_length=512),
	FieldSchema('description', dtype=DataType.VARCHAR, max_length=1024),
	
	FieldSchema('style', dtype=DataType.ARRAY, element_type=DataType.VARCHAR, max_capacity=32, max_length=128),
	FieldSchema('occasion', dtype=DataType.ARRAY, element_type=DataType.VARCHAR, max_capacity=32, max_length=128),
	FieldSchema('season', dtype=DataType.ARRAY, element_type=DataType.VARCHAR, max_capacity=32, max_length=128),
	FieldSchema('color', dtype=DataType.ARRAY, element_type=DataType.VARCHAR, max_capacity=32, max_length=128),
	FieldSchema('pattern', dtype=DataType.ARRAY, element_type=DataType.VARCHAR, max_capacity=32, max_length=128),
	FieldSchema('material', dtype=DataType.ARRAY, element_type=DataType.VARCHAR, max_capacity=32, max_length=128),
	FieldSchema('fit', dtype=DataType.ARRAY, element_type=DataType.VARCHAR, max_capacity=32, max_length=128),
	FieldSchema('length', dtype=DataType.ARRAY, element_type=DataType.VARCHAR, max_capacity=32, max_length=128),
	FieldSchema('neckline', dtype=DataType.ARRAY, element_type=DataType.VARCHAR, max_capacity=32, max_length=128),
	FieldSchema('sleeve_types', dtype=DataType.ARRAY, element_type=DataType.VARCHAR, max_capacity=32, max_length=128),
	FieldSchema('silhouette', dtype=DataType.ARRAY, element_type=DataType.VARCHAR, max_capacity=32, max_length=128),
	FieldSchema('texture', dtype=DataType.ARRAY, element_type=DataType.VARCHAR, max_capacity=32, max_length=128),
	FieldSchema('functionality', dtype=DataType.ARRAY, element_type=DataType.VARCHAR, max_capacity=32, max_length=128),
	FieldSchema('closure_type', dtype=DataType.ARRAY, element_type=DataType.VARCHAR, max_capacity=32, max_length=128),
	FieldSchema('embellishments', dtype=DataType.ARRAY, element_type=DataType.VARCHAR, max_capacity=32, max_length=128),
	FieldSchema('hemline', dtype=DataType.ARRAY, element_type=DataType.VARCHAR, max_capacity=32, max_length=128),

	FieldSchema('description_embedding', dtype=DataType.VARCHAR, max_length=4096),
	FieldSchema('embedding', dtype=DataType.FLOAT_VECTOR, dim=4096),
]
schema = CollectionSchema(
	fields,
	enable_dynamic_field=True,
)

# # 2. Create the collection.
mc = Collection('clothes')

# 3. Index the collection.
# mc.create_index(
#  field_name='embedding',
#  index_params={
# 		'index_type': 'AUTOINDEX',
# 		'metric_type': 'COSINE',
# 	}
# )
for i in range(0, 200):
	try:
		insert_result = mc.insert([clothes200[i]])
	except Exception: 
		print(i)
# After final entity is inserted, call flush 
# to stop growing segments left in memory.
mc.flush() 
print(mc.partitions)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
[{"name":"_default","collection_name":"clothes","description":""}]


In [10]:
collection = Collection('clothes')  # Get an existing collection.
collection.load()

In [11]:
# Query to get all 'id' values
ids = collection.query(expr="", output_fields=["id"], limit=1000)

# Extract the 'id' values from the query result
id_list = [item['id'] for item in ids]

print(id_list)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 220, 221,

# Testing

In [12]:
import random
import ipywidgets as widgets
from IPython.display import display, clear_output

In [13]:
preference = np.array([0 for _ in range(4096)])
already_seen = []

In [14]:
def search(query_vector, scalar_filter):
#  search_params = {
#  'metric_type': 'COSINE',
#  'params': {'nprobe': 10}
#  }
	return collection.search(
		data=[query_vector],
		anns_field='embedding',
		param={}, 
		output_fields=['id', 'name', 'imgs', 'description', 'price', 'url','embedding'],
		limit=1,
		expr=scalar_filter
	 )

In [17]:
# Function to display images and set up buttons
def generate_scalar_filter(selected_item):
	categories = [
		'style', 'occasion', 'season', 'color', 'pattern', 'material', 'fit', 
		'length', 'neckline', 'sleeve_types', 'silhouette', 'texture', 
		'functionality', 'closure_type', 'embellishments', 'hemline', 'clothing_category'
	]
	filters = []
	print(selected_item)
	for category in categories:
		print(category)
		values = selected_item.get(category)
		if values:
			chance = random.random()
			if chance < 0.05:
				# 5% chance to exclude any of the values
				for value in values:
					filters.append(f'{category} != "{value}"')
			elif chance < 0.50:
				# 45% chance to include at least one of the values
				values_str = ', '.join([f'"{value}"' for value in values])
				filters.append(f'{category} in [{values_str}]')
			# 50% chance to have no restriction
	return ' and '.join(filters) if filters else ''

# Function to handle button click
def on_button_click(like):
	global results
	global already_seen
	global preference

	selected_item = results[0][0].entity
	embedding = np.array(selected_item.get('embedding'))

	if like:
		preference = preference * 0.9 + 0.1 * embedding
	else:
		preference = preference - 0.1 * embedding

	norm = np.linalg.norm(preference)
	if norm != 0:
		preference = preference / norm

	already_seen.append(selected_item.get('id'))

	scalar_filter = generate_scalar_filter(selected_item)
	if scalar_filter:
		scalar_filter = f'({scalar_filter}) and id not in {already_seen}'
	else:
		scalar_filter = f'id not in {already_seen}'
	print(scalar_filter)
	# Update the results and display the next image
	results = search(list(preference), scalar_filter)
	display_images(results)

# Function to display images and set up buttons
def display_images(res):
	clear_output(wait=True)
	img = widgets.Image(
		value=requests.get(nakd_url + res[0][0].entity.get('imgs')[0]).content, 
		format='png', 
		layout=widgets.Layout(width='512px', height='512px')
	)
	name = widgets.Label(value=res[0][0].entity.get('name'))

	button_like = widgets.Button(description='Like')
	button_dislike = widgets.Button(description='Dislike')

	# Center the buttons under the image
	vbox = widgets.VBox([img, name, button_like, button_dislike], layout=widgets.Layout(align_items='center'))

	display(vbox)

	button_like.on_click(lambda x: on_button_click(True))
	button_dislike.on_click(lambda x: on_button_click(False))

# Initialize and display the first pair of images
already_seen = []
results = search(list(preference), '')
display_images(results)